# Import Dependencies

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import chardet

# Extract: Import files/create dataframes

In [3]:
#------------------------Read CSV Arrival Data----------------------------#
#(This file contains number of arrivals by travelers, per year up to 2018)#

arrivals_file = "resources/data/travel/API_ST.INT.ARVL_DS2_en_csv_v2_10515875.csv"
arrivals_df = pd.read_csv(arrivals_file , encoding = 'latin-1')
arrivals_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,...,813000.0,824000.0,869000.0,904000.0,979000.0,1072000.0,1225000.0,1102000.0,1070500.0,NaN
1,Afghanistan,AFG,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,...,366000.0,425000.0,481000.0,528000.0,650000.0,595000.0,592000.0,397000.0,261000.0,NaN
3,Albania,ALB,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,...,1711000.0,2191000.0,2469000.0,3156000.0,2857000.0,3341000.0,3784000.0,4070000.0,4643000.0,NaN
4,Andorra,AND,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,...,1830000.0,1808000.0,2242000.0,2238000.0,2328000.0,2363000.0,2663000.0,2831000.0,NaN,NaN


In [4]:
#----------------------------------Read CSV metadata file-----------------------------------------#
#(This file contains country code column, and some demographic info such as region & income group)#

metadata_file =  "resources/data/travel/Metadata_Country_API_ST.INT.ARVL_DS2_en_csv_v2_10515875.csv"
metadata_df = pd.read_csv(metadata_file, encoding='utf-8')
metadata_df.head()

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,Mining is included in agriculture\nElectricty ...,Aruba,NaN
1,AFG,South Asia,Low income,Fiscal year end: March 20; reporting period fo...,Afghanistan,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,NaN,Angola,NaN
3,ALB,Europe & Central Asia,Upper middle income,NaN,Albania,NaN
4,AND,Europe & Central Asia,High income,WB-3 code changed from ADO to AND to align wit...,Andorra,NaN


In [5]:
#------------------------Read CSV metadata file-----------------------------#
#(This file contains region, population, area size, GDP, mortality and more)#
happiness_file = "resources/data/happiness/2017_Happiness.csv"
happiness_df = pd.read_csv(happiness_file)
happiness_df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [6]:
#Extract happiness index JSON file and read in. Note that Coastline contains a dict within a dict
json_file = "resources/data/happiness/convertcsv.json"
country_facts_df = pd.read_json(json_file)
country_facts_df.head()

,Agriculture,Arable (%),Area (sq. mi.),Birthrate,Climate,Coastline (coast,Country,Crops (%),Deathrate,GDP ($ per capita),Industry,Infant mortality (per 1000 births),Literacy (%),Net migration,Other (%),Phones (per 1000),Pop. Density (per sq. mi.),Population,Region,Service
0,0.380,12.13,647500,46.60,1.0,{'area ratio)': 0},Afghanistan,0.22,20.34,700.0,0.240,163.07,36.0,23.06,87.65,3.2,48.0,31056997,ASIA (EX. NEAR EAST),0.380
1,0.232,21.09,28748,15.11,3.0,{'area ratio)': 1.26},Albania,4.42,5.22,4500.0,0.188,21.52,86.5,-4.93,74.49,71.2,124.6,3581655,EASTERN EUROPE,0.579
2,0.101,3.22,2381740,17.14,1.0,{'area ratio)': 0.04},Algeria,0.25,4.61,6000.0,0.600,31.00,70.0,-0.39,96.53,78.1,13.8,32930091,NORTHERN AFRICA,0.298
3,NaN,10.00,199,22.46,2.0,{'area ratio)': 58.29},American Samoa,15.00,3.27,8000.0,NaN,9.27,97.0,-20.71,75.00,259.5,290.4,57794,OCEANIA,NaN
4,NaN,2.22,468,8.71,3.0,{'area ratio)': 0},Andorra,0.00,6.25,19000.0,NaN,4.05,100.0,6.60,97.78,497.2,152.1,71201,WESTERN EUROPE,NaN


In [7]:
#---------------------Read CSV birthrate file---------------------------#
csv_file = "resources/data/birthrate/crude_birthrate.csv"
birthrate_df = pd.read_csv(csv_file)
birthrate_df.head()

,Series Name,Series Code,Country Name,Country Code,2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Afghanistan,AFG,33.214,..,..
1,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Albania,ALB,11.816,..,..
2,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Algeria,DZA,23.132,..,..
3,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,American Samoa,ASM,..,..,..
4,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Andorra,AND,8.8,..,..


# Transform: Clean DataFrames

In [8]:
#------------Arrivals dataframe------------#

# First list columns and keep only columns we are interested in
list(arrivals_df)
new_arrivals_df = arrivals_df[['Country Name','Country Code','2008','2009','2010','2011','2012','2013','2014',\
 '2015','2016','2017','2018']]
new_arrivals_df.head()

# Noticed on inspection that there is no data for 2018 so will drop column
print(new_arrivals_df['2018'])
del new_arrivals_df['2018']
new_arrivals_df.head()

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
24    NaN
25    NaN
26    NaN
27    NaN
28    NaN
29    NaN
       ..
234   NaN
235   NaN
236   NaN
237   NaN
238   NaN
239   NaN
240   NaN
241   NaN
242   NaN
243   NaN
244   NaN
245   NaN
246   NaN
247   NaN
248   NaN
249   NaN
250   NaN
251   NaN
252   NaN
253   NaN
254   NaN
255   NaN
256   NaN
257   NaN
258   NaN
259   NaN
260   NaN
261   NaN
262   NaN
263   NaN
Name: 2018, Length: 264, dtype: float64


,Country Name,Country Code,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Aruba,ABW,827000.0,813000.0,824000.0,869000.0,904000.0,979000.0,1072000.0,1225000.0,1102000.0,1070500.0
1,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,294000.0,366000.0,425000.0,481000.0,528000.0,650000.0,595000.0,592000.0,397000.0,261000.0
3,Albania,ALB,1247000.0,1711000.0,2191000.0,2469000.0,3156000.0,2857000.0,3341000.0,3784000.0,4070000.0,4643000.0
4,Andorra,AND,2059000.0,1830000.0,1808000.0,2242000.0,2238000.0,2328000.0,2363000.0,2663000.0,2831000.0,NaN
